In [6]:
!python3 -m pip install wget
import wget
!python3 -m wget https://github.com/mmanzato/6EABDARecSys/raw/main/dataset/AmazonMusic.tar.xz
!tar -xf AmazonMusic.tar.xz
!python3 -m pip install caserecommender

100% [....................................................] 23861964 / 23861964
Saved under AmazonMusic.tar.xz
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python3.8 -m pip install --upgrade pip


In [7]:
import numpy as np
import pandas as pd

In [8]:
ls -l

total 60080
-rw-rw-r--@ 1 manzato  staff   4645392 Sep 29 11:09 4. Filtragem Colaborativa.pptx
drwxr-xr-x  6 manzato  staff       192 Oct 13  2020 AmazonMusic/
-rw-r--r--  1 manzato  staff  23861964 Sep 29 16:18 AmazonMusic.tar.xz
-rw-rw-r--@ 1 manzato  staff     21761 Sep 29 16:17 FC.ipynb
-rw-r--r--  1 manzato  staff     74632 Jul 18  2020 dict_title.npy
-rw-r--r--  1 manzato  staff    291777 Oct 13  2020 test.txt
-rw-r--r--  1 manzato  staff   1177686 Oct 13  2020 train.txt


In [12]:
dataset = pd.read_json('./AmazonMusic/Digital_Music_5.json', lines=True)
dataset_metadata = pd.read_csv('./AmazonMusic/amazon_music_metadata.csv')

df = dataset[['reviewerID', 'asin', 'overall']]
df = df.merge(dataset_metadata[['asin', 'title']])

map_users = {user: idx for idx, user in enumerate(df.reviewerID.unique())}
map_items = {item: idx for idx, item in enumerate(df.asin.unique())}
df['reviewerID'] = df['reviewerID'].map(map_users)
df['asin'] = df['asin'].map(map_items)
    
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

train.to_csv('train.txt', index=False, header=False, sep='\t')
test.to_csv('test.txt', index=False, header=False, sep='\t')

In [18]:
map_title = {}

for _, row in df.iterrows():
    map_title[row.asin] = row.title

In [13]:
test = pd.read_csv('test.txt', sep='\t', names=['reviewerID', 'asin', 
                                                'rate', 'title'])
test[test.reviewerID == 0]

,reviewerID,asin,rate,title
4870,0,0,5,Memory of Trees
9411,0,959,5,Ray of Light


## Memory-based

In [14]:
from caserec.recommenders.rating_prediction.itemknn import ItemKNN

ItemKNN('train.txt', 'test.txt', 'rp_iknn.dat').compute()

[Case Recommender: Rating Prediction > ItemKNN Algorithm]

train data:: 5099 users and 2582 items (41436 interactions) | sparsity:: 99.69%
test data:: 3818 users and 2288 items (10360 interactions) | sparsity:: 99.88%

training_time:: 6.111202 sec
prediction_time:: 0.423604 sec
Eval:: MAE: 0.698814 RMSE: 1.085571 


In [20]:
predictions = pd.read_csv('rp_iknn.dat', sep='\t', names=['reviewerID', 'asin', 'rate'])
predictions['title'] = predictions.asin.map(map_title)
predictions.head(3)

,reviewerID,asin,rate,title
0,0,0,2.507352,Memory of Trees
1,0,959,2.502669,Ray of Light
2,1,667,4.931571,For the Roses


In [16]:
from caserec.recommenders.rating_prediction.userknn import UserKNN

UserKNN('train.txt', 'test.txt', 'rp_uknn.dat').compute()

[Case Recommender: Rating Prediction > UserKNN Algorithm]

train data:: 5099 users and 2582 items (41436 interactions) | sparsity:: 99.69%
test data:: 3818 users and 2288 items (10360 interactions) | sparsity:: 99.88%

training_time:: 6.052077 sec
prediction_time:: 2.117171 sec
Eval:: MAE: 0.673922 RMSE: 0.974793 


In [19]:
predictions = pd.read_csv('rp_uknn.dat', sep='\t', names=['reviewerID', 'asin', 'rate'])
predictions['title'] = predictions.asin.map(map_title)
predictions.head(3)

,reviewerID,asin,rate,title
0,0,0,4.967764,Memory of Trees
1,0,959,4.085450,Ray of Light
2,1,667,5.000000,For the Roses


## Model-based

In [21]:
from caserec.recommenders.rating_prediction.matrixfactorization import MatrixFactorization

MatrixFactorization('train.txt', 'test.txt', 'rp_mf.dat', factors=3).compute()

[Case Recommender: Rating Prediction > Matrix Factorization]

train data:: 5099 users and 2582 items (41436 interactions) | sparsity:: 99.69%
test data:: 3818 users and 2288 items (10360 interactions) | sparsity:: 99.88%

training_time:: 0.923295 sec
prediction_time:: 0.024067 sec


Eval:: MAE: 0.837403 RMSE: 1.075602 


In [22]:
from caserec.recommenders.rating_prediction.svd import SVD

SVD('train.txt', 'test.txt', 'rp_mf.dat', factors=30).compute()

[Case Recommender: Rating Prediction > SVD]

train data:: 5099 users and 2582 items (41436 interactions) | sparsity:: 99.69%
test data:: 3818 users and 2288 items (10360 interactions) | sparsity:: 99.88%

training_time:: 1.231262 sec
prediction_time:: 0.011789 sec


Eval:: MAE: 0.815141 RMSE: 1.127661 
